In [ ]:
from PyQt5 import QtCore, QtGui, QtWidgets
from tkinter import *
import cv2
from PyQt5.QtWidgets import *
import sys
import time
import tkinter as tk
from tkinter import filedialog, messagebox
import socket
import numpy as np
import dlib
import imutils
from imutils.face_utils import *
import datetime
from glob import glob
import os
from PIL import Image
import paramiko
# from PyQt5.QtMultimedia import *
# from PyQt5 import QtMultimediaWidgets

In [ ]:
# 取得預設的臉部偵測器
detector = dlib.get_frontal_face_detector()
# 根據shape_predictor方法載入68個特徵點模型，此方法為人臉表情識別的偵測器
landmark_predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat') 

In [ ]:
#伺服器資訊
HOST = '140.116.39.114'
PORT = 8002

# s = socket.socket(socket.AF_INET,socket.SOCK_STREAM)#創建socket
# s.connect((HOST, PORT))

In [ ]:
def videotolip(file_path, content, theTime):
    capture = cv2.VideoCapture(file_path)
    folder_name = "./AddData/"+str(theTime)
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
        if not os.path.isdir(folder_name+"/lip"):
            os.mkdir(folder_name+"/lip") 
    with open(folder_name + "/text.txt" , 'w', encoding='utf-8') as txt_data:
        txt_data.write(content)
#         txt_data.write('\n')
    count = 1
    while(capture.isOpened()):
        ret, frame = capture.read()
#         time.sleep(0.005) 
        if ret == True:
            face_rects = detector(frame, 1)

            for i, d in enumerate(face_rects):
                x1 = d.left()
                y1 = d.top()
                x2 = d.right()
                y2 = d.bottom()

                if i == 0:
                    shape = landmark_predictor(frame, d) 
                    # 將特徵點轉為numpy (68,2)
                    shape = shape_to_np(shape)

                    # loop over the face parts individually 
                    for(name, (i,j)) in FACIAL_LANDMARKS_IDXS.items(): 
                        if i==48 and j==68:
                            clone = frame.copy() 
                            # extract the ROI of the face region as a separate image 
                            # x，y是矩阵左上点的坐标，w，h是矩阵的宽和高
                            (x, y, w, h) = cv2.boundingRect(np.array([shape[i:j]])) 
                            y = y-10
                            x = x-10
                #             print(x, y, w, h)
                            roi = frame[y : y + h + 20, x: x + w + 20]
                            roi = imutils.resize(roi, width = 100, inter = cv2.INTER_CUBIC)
                            cv2.imwrite(folder_name+"/lip/"+str(count)+".jpg", roi)
                            count = count + 1
        else:
            break

In [ ]:
def LiptoNumpy(theTime):
    maxlen = 77
    image_list = []
    filesPath = "./AddData/" + str(theTime) + "/lip/*"
    lists1 = glob(filesPath)
    lists1.sort()
    lists1 = lists1[10:-3]         # 去除前10張和後3張 
    if len(lists1) <= maxlen:
        for img_path in lists1:        
            image = Image.open(img_path)

            image = image.resize((100,50), Image.BILINEAR) 
            np_image = np.array(image)
            image_list.append(np_image)

        if len(lists1) != maxlen:
            zeros = np.zeros((maxlen-len(lists1),50,100,3)).astype(int)
            image_list = np.append(image_list, zeros, axis=0)
#             image_list = np.concatenate((image_list, zeros), axis=0)
            image_list = image_list.tolist()
    # save npy file
    np.save('./AddData/' + str(theTime) + "/np", np.array(image_list))

In [ ]:
def transferFile(theTime, content):
    scp = paramiko.Transport(("140.116.39.114", 22))
    scp.connect(username="eagleuser", password="nckulina41504eagleepson")
    sftp = paramiko.SFTPClient.from_transport(scp)
    
    remote_path = "/home/eagleuser/Users/dragon/Chinese_vocab_LipReading/system_data/transfer_data/" + str(theTime)
    try:
        sftp.chdir(remote_path) # Test if remote_path exists
    except IOError:
        sftp.mkdir(remote_path) # Create remote_path
        sftp.chdir(remote_path)
    
    localpath = "D:\\jupyter\\Chinese Vocab LipReading\\system\\AddData\\" + theTime + "\\np.npy"
    filepath = "/home/eagleuser/Users/dragon/Chinese_vocab_LipReading/system_data/transfer_data/" + theTime + "/np.npy"
    sftp.put(localpath, filepath) # upload 檔案
    localpath1 = "D:\\jupyter\\Chinese Vocab LipReading\\system\\AddData\\" + theTime + "\\text.txt"
    filepath1 = "/home/eagleuser/Users/dragon/Chinese_vocab_LipReading/system_data/transfer_data/" + theTime + "/text.txt"
    sftp.put(localpath1, filepath1) # upload 檔案
    
    sftp.close()
    scp.close()

In [ ]:
def PredictResult(file_path):
    capture = cv2.VideoCapture(file_path)
    theTime = time.strftime("%a %b %d %H:%M:%S %Y", time.localtime())
    theTime = time.mktime(time.strptime(theTime,"%a %b %d %H:%M:%S %Y"))
    folder_name = "./PredictData/"+ str(theTime)
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
        if not os.path.isdir(folder_name+"/lip"):
            os.mkdir(folder_name+"/lip") 
    
    count = 1
    while(capture.isOpened()):
        ret, frame = capture.read()
#         time.sleep(0.005) 
        if ret == True:
            face_rects = detector(frame, 1)

            for i, d in enumerate(face_rects):
                x1 = d.left()
                y1 = d.top()
                x2 = d.right()
                y2 = d.bottom()

                if i == 0:
                    shape = landmark_predictor(frame, d) 
                    # 將特徵點轉為numpy (68,2)
                    shape = shape_to_np(shape)

                    # loop over the face parts individually 
                    for(name, (i,j)) in FACIAL_LANDMARKS_IDXS.items(): 
                        if i==48 and j==68:
                            clone = frame.copy() 
                            # extract the ROI of the face region as a separate image 
                            # x，y是矩阵左上点的坐标，w，h是矩阵的宽和高
                            (x, y, w, h) = cv2.boundingRect(np.array([shape[i:j]])) 
                            y = y-10
                            x = x-10
                #             print(x, y, w, h)
                            roi = frame[y : y + h + 20, x: x + w + 20]
                            roi = imutils.resize(roi, width = 100, inter = cv2.INTER_CUBIC)
                            cv2.imwrite(folder_name+"/lip/"+str(count)+".jpg", roi)
                            count = count + 1
        else:
            break
    
    maxlen = 77
#     image_list = []
    filesPath = "./PredictData/" + str(theTime) + "/lip/*"
    lists1 = glob(filesPath)
    lists1.sort()
    lists1 = lists1[10:-3]         # 去除前10張和後3張 
    
    num = 0
    split = 15                     # 決定多少個frame作切割
    for i in range(0, len(lists1), split):
        lists2 = lists1[i:i+split]
        image_list = []
        num = num + 1
        if len(lists2) <= maxlen:
            for img_path in lists2:        
                image = Image.open(img_path)

                image = image.resize((100,50), Image.BILINEAR) 
                np_image = np.array(image)
                image_list.append(np_image)

            if len(lists2) != maxlen:
                zeros = np.zeros((maxlen-len(lists2),50,100,3)).astype(int)
                image_list = np.append(image_list, zeros, axis=0)
    #             image_list = np.concatenate((image_list, zeros), axis=0)
                image_list = image_list.tolist()
        # save npy file
        np.save('./PredictData/' + str(theTime) + "/" + str(num), np.array(image_list))

        scp = paramiko.Transport(("140.116.39.114", 22))
        scp.connect(username="eagleuser", password="nckulina41504eagleepson")
        sftp = paramiko.SFTPClient.from_transport(scp)

        remote_path = "/home/eagleuser/Users/dragon/Chinese_vocab_LipReading/system_data/predict_data/" + str(theTime)
        try:
            sftp.chdir(remote_path) # Test if remote_path exists
        except IOError:
            sftp.mkdir(remote_path) # Create remote_path
            sftp.chdir(remote_path)

        localpath = "D:\\jupyter\\Chinese Vocab LipReading\\system\\PredictData\\" + str(theTime) + "\\" + str(num) + ".npy"
        filepath = "/home/eagleuser/Users/dragon/Chinese_vocab_LipReading/system_data/predict_data/" + str(theTime) + "/" + str(num) + ".npy"
        sftp.put(localpath, filepath) # upload 檔案

        sftp.close()
        scp.close()
    return(str(theTime))

In [ ]:
class Ui_MainWindow(object):
    def __init__(self):
        self.setupUi(MainWindow)       # 执行类中的setupUi方法，方法的参数是第二步中创建的QMainWindow
        self.file_path = ""
        root = tk.Tk()
        root.withdraw()

    def setupUi(self, MainWindow):
        MainWindow.setObjectName("MainWindow")
        MainWindow.resize(611, 450)
        
        self.centralwidget = QtWidgets.QWidget(MainWindow)
        self.centralwidget.setObjectName("centralwidget")
        
        self.pushButton_video = QtWidgets.QPushButton(self.centralwidget)
        self.pushButton_video.setGeometry(QtCore.QRect(250, 212, 91, 21))
        self.pushButton_video.setObjectName("pushButton_video")
        self.pushButton_video.clicked.connect(self.display_video)
        # ------------------------------------------------------------------------------------------
        self.verticalLayoutWidget = QtWidgets.QWidget(self.centralwidget)
        self.verticalLayoutWidget.setGeometry(QtCore.QRect(0, 0, 611, 201))
        self.verticalLayoutWidget.setObjectName("verticalLayoutWidget")
        self.verticalLayout = QtWidgets.QVBoxLayout(self.verticalLayoutWidget)
        self.verticalLayout.setContentsMargins(0, 0, 0, 0)
        self.verticalLayout.setObjectName("verticalLayout")
        # ------------------------------------------------------------------------------------------
        self.horizontalLayoutWidget = QtWidgets.QWidget(self.centralwidget)
        self.horizontalLayoutWidget.setGeometry(QtCore.QRect(110, 240, 371, 164))
        self.horizontalLayoutWidget.setObjectName("horizontalLayoutWidget")
        self.horizontalLayout = QtWidgets.QHBoxLayout(self.horizontalLayoutWidget)
        self.horizontalLayout.setContentsMargins(0, 0, 0, 0)
        self.horizontalLayout.setObjectName("horizontalLayout")
        # ------------------------------------------------------------------------------------------
        self.verticalLayout_2 = QtWidgets.QVBoxLayout()
        self.verticalLayout_2.setObjectName("verticalLayout_2")
        
        self.label = QtWidgets.QLabel(self.horizontalLayoutWidget)
        font = QtGui.QFont()
        font.setFamily("Agency FB")
        font.setPointSize(16)
        self.label.setFont(font)
        self.label.setWordWrap(False)
        self.label.setObjectName("label")
        self.verticalLayout_2.addWidget(self.label)
        
        self.textEdit = QtWidgets.QTextEdit(self.horizontalLayoutWidget)
        self.textEdit.setObjectName("textEdit")
        self.verticalLayout_2.addWidget(self.textEdit)
        
        self.pushButton_add = QtWidgets.QPushButton(self.horizontalLayoutWidget)
        self.pushButton_add.setObjectName("pushButton_add")
        self.pushButton_add.clicked.connect(self.add_data)
        self.verticalLayout_2.addWidget(self.pushButton_add)
        
        self.pushButton_train = QtWidgets.QPushButton(self.horizontalLayoutWidget)
        self.pushButton_train.setObjectName("pushButton_train")
        self.pushButton_train.clicked.connect(self.train_data)
        self.verticalLayout_2.addWidget(self.pushButton_train)
        
        self.horizontalLayout.addLayout(self.verticalLayout_2)
        # ------------------------------------------------------------------------------------------
        self.verticalLayout_3 = QtWidgets.QVBoxLayout()
        self.verticalLayout_3.setObjectName("verticalLayout_3")
        
        self.label_2 = QtWidgets.QLabel(self.horizontalLayoutWidget)
        font = QtGui.QFont()
        font.setFamily("Agency FB")
        font.setPointSize(16)
        self.label_2.setFont(font)
        self.label_2.setObjectName("label_2")
        self.verticalLayout_3.addWidget(self.label_2)
        
        self.pushButton_predict = QtWidgets.QPushButton(self.horizontalLayoutWidget)
        self.pushButton_predict.setObjectName("pushButton_predict")
        self.pushButton_predict.clicked.connect(self.predict_result)
        self.verticalLayout_3.addWidget(self.pushButton_predict)
        
        self.textEdit_2 = QtWidgets.QTextEdit(self.horizontalLayoutWidget)
        self.textEdit_2.setObjectName("textEdit_2")
        self.verticalLayout_3.addWidget(self.textEdit_2)
        self.horizontalLayout.addLayout(self.verticalLayout_3)
        # ------------------------------------------------------------------------------------------
        MainWindow.setCentralWidget(self.centralwidget)
        self.menubar = QtWidgets.QMenuBar(MainWindow)
        self.menubar.setGeometry(QtCore.QRect(0, 0, 611, 21))
        self.menubar.setObjectName("menubar")
        MainWindow.setMenuBar(self.menubar)
        self.statusbar = QtWidgets.QStatusBar(MainWindow)
        self.statusbar.setObjectName("statusbar")
        MainWindow.setStatusBar(self.statusbar)

        self.retranslateUi(MainWindow)
        QtCore.QMetaObject.connectSlotsByName(MainWindow)

    def retranslateUi(self, MainWindow):
        _translate = QtCore.QCoreApplication.translate
        MainWindow.setWindowTitle(_translate("MainWindow", "MainWindow"))
        self.pushButton_video.setText(_translate("MainWindow", "choose video"))
        self.label.setText(_translate("MainWindow", "擴增資料"))
        self.pushButton_add.setText(_translate("MainWindow", "擴增"))
        self.pushButton_train.setText(_translate("MainWindow", "訓練"))
        self.label_2.setText(_translate("MainWindow", "辨識"))
        self.pushButton_predict.setText(_translate("MainWindow", "預測結果"))
        
    def display_video(self):
        self.file_path = filedialog.askopenfilename(initialdir="D:\\jupyter\\Chinese Vocab LipReading", title="Select file", filetypes = (("MP4 files","*.mp4"),("all files","*.*")))
#         print(file_path)
        capture = cv2.VideoCapture(self.file_path)
        # 以迴圈從影片檔案讀取影格，並顯示出來
        while(capture.isOpened()):
            ret, frame = capture.read()
#             time.sleep(0.005) 
            if ret == True:
                frame = cv2.resize(frame,(600,200)) 
                cv2.imshow('frame',frame)
            else:
                break
            k = cv2.waitKey(10) & 0xff
            if k == 27:                                           #等待 ESC 鍵
                break
        capture.release()
        cv2.destroyAllWindows()

    def add_data(self):
        content = self.textEdit.toPlainText()                           # 讀取text中的內容
        if self.file_path != "" and content != "":
            theTime = time.strftime("%a %b %d %H:%M:%S %Y", time.localtime())
            theTime = time.mktime(time.strptime(theTime,"%a %b %d %H:%M:%S %Y"))
#             theTime = "1590873575.0"

            videotolip(self.file_path, content, str(theTime))
            LiptoNumpy(str(theTime))
            transferFile(str(theTime), str(content))
            
            messagebox.showinfo("Information","資料新增完成!!!")
            self.file_path = ""
        else:
            messagebox.showinfo("Information","請選擇影片並填寫標籤!!!")
           
    def train_data(self):
        MsgBox = messagebox.askquestion ('Warning','確定要重新訓練嗎？',icon = 'warning')
        if MsgBox == 'yes':
            s = socket.socket(socket.AF_INET,socket.SOCK_STREAM)#創建socket
            s.connect((HOST, PORT))
            mode = "train"
            s.send(mode.encode())
            
            msg = str(s.recv(1024), encoding='utf-8')
            messagebox.showinfo("Information", msg)
            s.close()
        else:
            pass
        
    def predict_result(self):
        if self.file_path != "":
            s = socket.socket(socket.AF_INET,socket.SOCK_STREAM)#創建socket
            s.connect((HOST, PORT))
            mode = "predict"
            s.send(mode.encode())
            
            theTime = PredictResult(self.file_path)
            s.send(theTime.encode())
            
            result = str(s.recv(1024), encoding='utf-8')
            
            self.textEdit_2.setPlainText(result)       # 輸出在text中
            self.file_path = ""
            messagebox.showinfo("Information", "predict finish!!!")
            s.close()
        else:
            messagebox.showinfo("Information","請選擇要預測的影片!!!")

In [ ]:
if __name__ == "__main__":
    app = QtWidgets.QApplication(sys.argv)  # 创建一个QApplication，也就是你要开发的软件app
    MainWindow = QtWidgets.QMainWindow()    # 创建一个QMainWindow，用来装载你需要的各种组件、控件
    ui = Ui_MainWindow()                    # ui是Ui_MainWindow()类的实例化对象
#     ui.setupUi(MainWindow)                # 执行类中的setupUi方法，方法的参数是第二步中创建的QMainWindow
    MainWindow.show()                       # 执行QMainWindow的show()方法，显示这个QMainWindow
    sys.exit(app.exec_())                   # 使用exit()或者点击关闭按钮退出QApplication